In [1]:
import keras
#from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications.imagenet_utils import preprocess_input

from sklearn.utils import shuffle
import cv2
import glob
import os
import tensorflow as tf
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


ModuleNotFoundError: No module named 'cv2'

In [ ]:
normal_path = glob.glob("..\\chest-xray-pneumonia\\chest_xray\\train\\NORMAL_old\\*jpeg")
pneumonia_path = glob.glob("..\\chest-xray-pneumonia\\chest_xray\\train\\PNEUMONIA_old\\*jpeg")

#val
normal_path_val = glob.glob("..\\chest-xray-pneumonia\\chest_xray\\val\\NORMAL_old\\*jpeg")
pneumonia_path_val = glob.glob("..\\chest-xray-pneumonia\\chest_xray\\val\\PNEUMONIA_old\\*jpeg")

### test 
normal_path_test = glob.glob("..\\chest-xray-pneumonia\\chest_xray\\test\\NORMAL\\*jpeg")
pneumonia_path_test = glob.glob("..\\chest-xray-pneumonia\\chest_xray\\test\\PNEUMONIA\\*jpeg")

In [ ]:
normal_target = [0]*len(normal_path)
pneumonia_target = [1]*len(pneumonia_path)

target = normal_target + pneumonia_target
image_path = normal_path + pneumonia_path

normal_target_val = [0]*len(normal_path_val)
pneumonia_target_val = [1]*len(pneumonia_path_val)

target_val = normal_target_val + pneumonia_target_val
image_path_val = normal_path_val + pneumonia_path_val


normal_target_test = [0]*len(normal_path_test)
pneumonia_target_test = [1]*len(pneumonia_path_test)

target_test = normal_target_test + pneumonia_target_test
image_path_test = normal_path_test + pneumonia_path_test

### generator

In [ ]:
def generator(image_path,target,batch_size):
      
    d = {'0':[1.,0.],'1':[0.,1.]}
    size = len(image_path)
    
    image_path , target = shuffle (image_path,target)
    while 1:
        for offset in range(0,int(len(image_path)/batch_size)+0):
            train = []
            target_string = []
            end = offset*batch_size+batch_size
            if end >len(image_path):
                end = len(image_path)
                
                
            path = image_path[0+offset*batch_size:end]
            targ = target[0+offset*batch_size:end]
            for a,b in zip(path,targ):
                
                img = image.load_img(a, target_size=(224, 224))
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                img = preprocess_input(x)
                
                img = np.reshape(img,(224,224,3))
                
                if (type(img) !=bool) and (type(img) != None):
                        #print (img.shape)
                        train.append(img)
                        target_string.append(str(b))# works for one hot encoding
                        #target_string.append(b)
            targ_hot = []
            for a in target_string:
                targ_hot.append(d[a])# works for one hot encoding
                #targ_hot.append(float(a))
                
            #print("ooo",len(train[0]))
            X_train = np.array(train)
            Y_train = np.array(targ_hot)
            #print("shh",X_train.shape)
         
            yield shuffle(X_train,Y_train)

In [ ]:
batch_size = 8
training_generator = generator(image_path,target,batch_size)
val_generator = generator(image_path_val,target_val,batch_size)

In [2]:
import densenet_separable
image_dim = (224, 224, 3)
model = densenet_separable.DenseNetImageNet121(input_shape=image_dim,weights=None,classes=2)

The activation used here is :  softmax


In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 112, 112, 64) 339         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         separable_conv2d_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [4]:
from keras.optimizers import Adam

In [ ]:
optimizer = Adam(lr=0.001)
epochs = 100
file = 'weights//dense_softmax_2_non_local//weights.{epoch:02d}-{val_loss:.2f}.hdf5'
ckpt_save =keras.callbacks.ModelCheckpoint(file, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
ckpt_tensorboard = keras.callbacks.TensorBoard(log_dir='./logs_softmax_2_non_local', histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

ckpt_list = [ckpt_save,ckpt_tensorboard]
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[keras.metrics.binary_accuracy])

model.fit_generator(training_generator,epochs = epochs,callbacks = ckpt_list,validation_data = val_generator ,steps_per_epoch=len(image_path)/batch_size,validation_steps=len(image_path_val)/batch_size )